In [1]:
import numpy as np
import pandas as pd
import argparse
import os.path as osp

import torch
import torch.nn.functional as func
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score

import wandb
import yaml

from models.GCN import GCN
from dataloaders.GCN_loader import GraphDataset

# from dataloaders.batch_dataloader import FCMatrixDataset
from dataloaders.dataloader import FCMatrixDataset

from torch.utils.data import Dataset, Subset
from torch_geometric.loader import DataLoader


from utils import balanced_random_split_v2
from copy import deepcopy
from functools import partial

/home/ljdevries/.conda/envs/mridc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = "data/csv/severe_rds.csv"
data_dir = "data/fetched/25751"
mrmr_features = np.array([1140, 536, 223, 907, 1449, 499, 1293, 45, 135, 1440, 879, 1384, 1210, 1316, 122, 22, 492, 638, 765, 1027, 1464, 501, 1462, 395, 26, 1079, 70, 425, 1403, 1409, 1318, 886, 1459, 1448, 939, 1163, 547, 10, 413, 676, 131, 216, 942, 1136, 1386, 232, 1455, 1337, 814, 139, 392, 1376, 1382, 471, 656]
)
# sort mrmr features
mrmr_features = np.sort(mrmr_features)

input_features = 11

udi = data_dir.split("/")[-1]
# data_dir = data_dir
labels = np.genfromtxt(ds)
labels = labels[1:, 1]
dataset = GraphDataset(ds, data_dir, udi, None, mrmr = mrmr_features)
dataset2 = GraphDataset(ds, data_dir, udi, None, mrmr = None)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cpu")

data/csv/severe_rds.csv
data/csv/severe_rds.csv
data/csv/severe_rds.csv
data/csv/severe_rds.csv


In [3]:
model = GCN(input_features, 2, 3).to(DEVICE)
# checkpoint = torch.load("models/saved/MLP/decent-sweep.pth", map_location=DEVICE)
checkpoint = torch.load("models/saved/GCN/decent-sweep-36.pth", map_location=DEVICE)

model.load_state_dict(checkpoint)

model.conv1.lins


ModuleList(
  (0): Linear(11, 64, bias=False)
  (1): Linear(11, 64, bias=False)
  (2): Linear(11, 64, bias=False)
)

In [4]:
print(type(model))

<class 'models.GCN.GCN'>


In [5]:
from torch_geometric.data import Data, Batch

import os
os.environ["SHAP_FORCE_BACKEND"] = "pytorch"
import shap

# for i in dataset:
    # print(i)
    # exit()
# X = np.array([np.array(i) for i in dataset])
# print(X[:10].shape)
# X = torch.tensor(np.array([np.array(i) for i in dataset]))
# X_loader = DataLoader(X, batch_size=48, shuffle=True)
# X = DataLoader(dataset, batch_size=48, shuffle=True)
# print(X[0])

X = dataset
print(X[0].x)
X_loader = DataLoader(X, batch_size=48, shuffle=True)


# X_loader[0]


tensor([[ 0.0000, -2.0405,  2.9736, -1.9279, -2.0138, 15.2677, -2.7766, -3.7782,
         -0.1155,  0.1335, -4.8360],
        [-2.0405,  0.0000, -2.2577,  5.3103,  5.1687,  3.2211, -7.3531,  2.9986,
          2.1771,  2.6712,  1.8710],
        [ 2.9736, -2.2577,  0.0000,  1.7654, -8.7138,  6.9587, -7.5969, -6.7912,
         -0.7177,  0.1521,  7.4302],
        [-1.9279,  5.3103,  1.7654,  0.0000,  3.1087,  0.8900, -7.6152,  7.5293,
         -9.2442, -2.2570,  4.5055],
        [-2.0138,  5.1687, -8.7138,  3.1087,  0.0000,  7.7194,  9.6208,  4.6354,
          0.9936,  0.1609,  1.7838],
        [15.2677,  3.2211,  6.9587,  0.8900,  7.7194,  0.0000,  3.7219, -1.9091,
         -0.6371, -3.1070, -1.8756],
        [-2.7766, -7.3531, -7.5969, -7.6152,  9.6208,  3.7219,  0.0000,  2.5564,
         -3.1585,  4.1001, -2.0444],
        [-3.7782,  2.9986, -6.7912,  7.5293,  4.6354, -1.9091,  2.5564,  0.0000,
         -3.5454,  1.8930,  0.5323],
        [-0.1155,  2.1771, -0.7177, -9.2442,  0.9936, -0

In [6]:

loader = DataLoader(dataset, batch_size=48, shuffle=True)
B = next(iter(loader)) 


explainer = shap.DeepExplainer(model, B)
# explainer = shap.explainers.Sampling(model, B.x)
# # Explain predictions for a single instance

print(dataset[100:103].x.shape)
shap_values = explainer.shap_values(dataset)


torch.Size([33, 11])


AttributeError: 'GraphDataset' object has no attribute 'shape'

In [18]:
# from torch_geometric.data import Data, Batch

# B = Batch.from_data_list(X)

# explainer = shap.DeepExplainer(model, B)

# shap_values = explainer.shap_values(B)
# shap.summary_plot(shap_values)

AttributeError: 'GlobalStorage' object has no attribute 'shape'